In [1]:
from lightgbm.sklearn import LGBMRanker
import lightgbm as lgb
from datetime import timedelta
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import seaborn as sns
import gc

from datetime import datetime, date, timedelta

In [125]:
INPUT_DIR = '../input/'
OUTPUT_DIR = '../output/'
INTERMED_DIR = '../intermed/'

transactions = pd.read_csv(os.path.join(INPUT_DIR, 'transactions_train.csv'), dtype={"article_id": "str"})
customers = pd.read_csv(os.path.join(INPUT_DIR, 'customers.csv'))
articles = pd.read_csv(os.path.join(INPUT_DIR, 'articles.csv'), dtype={"article_id": "str"})

# Item

In [126]:
df = articles.drop(columns = ['product_type_name', 'graphical_appearance_name', 'colour_group_name', 'perceived_colour_value_name',
                        'perceived_colour_master_name', 'index_name', 'index_group_name', 'section_name', 
                        'garment_group_name', 'prod_name', 'department_name', 'detail_desc'])
df.head()

,article_id,product_code,product_type_no,product_group_name,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,department_no,index_code,index_group_no,section_no,garment_group_no
0,0108775015,108775,253,Garment Upper body,1010016,9,4,5,1676,A,1,16,1002
1,0108775044,108775,253,Garment Upper body,1010016,10,3,9,1676,A,1,16,1002
2,0108775051,108775,253,Garment Upper body,1010017,11,1,9,1676,A,1,16,1002
3,0110065001,110065,306,Underwear,1010016,9,4,5,1339,B,1,61,1017
4,0110065002,110065,306,Underwear,1010016,10,3,9,1339,B,1,61,1017


In [127]:
agg_df = transactions.groupby('article_id')[['price']].agg(['mean', 'max'])
agg_df.columns = ['price_mean', 'price_max']
agg_df.reset_index(drop=False, inplace=True)
agg_df.head()

,article_id,price_mean,price_max
0,0108775015,0.008142,0.009153
1,0108775044,0.008114,0.008508
2,0108775051,0.004980,0.008458
3,0110065001,0.020219,0.025407
4,0110065002,0.018205,0.025407


In [128]:
df = df.merge(agg_df, on='article_id', how='left')
df.head()

,article_id,product_code,product_type_no,product_group_name,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,department_no,index_code,index_group_no,section_no,garment_group_no,price_mean,price_max
0,0108775015,108775,253,Garment Upper body,1010016,9,4,5,1676,A,1,16,1002,0.008142,0.009153
1,0108775044,108775,253,Garment Upper body,1010016,10,3,9,1676,A,1,16,1002,0.008114,0.008508
2,0108775051,108775,253,Garment Upper body,1010017,11,1,9,1676,A,1,16,1002,0.004980,0.008458
3,0110065001,110065,306,Underwear,1010016,9,4,5,1339,B,1,61,1017,0.020219,0.025407
4,0110065002,110065,306,Underwear,1010016,10,3,9,1339,B,1,61,1017,0.018205,0.025407


In [129]:
transactions.sales_channel_id = transactions.sales_channel_id -1
agg_df = transactions.groupby('article_id')[['sales_channel_id']].mean()
agg_df.columns = ['mean_sales_channel_id']
agg_df.reset_index(drop=False, inplace=True)
agg_df.head()

,article_id,mean_sales_channel_id
0,0108775015,0.770778
1,0108775044,0.710207
2,0108775051,0.995349
3,0110065001,0.375479
4,0110065002,0.654917


In [130]:
df = df.merge(agg_df, on='article_id', how='left')
df.head()

,article_id,product_code,product_type_no,product_group_name,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,department_no,index_code,index_group_no,section_no,garment_group_no,price_mean,price_max,mean_sales_channel_id
0,0108775015,108775,253,Garment Upper body,1010016,9,4,5,1676,A,1,16,1002,0.008142,0.009153,0.770778
1,0108775044,108775,253,Garment Upper body,1010016,10,3,9,1676,A,1,16,1002,0.008114,0.008508,0.710207
2,0108775051,108775,253,Garment Upper body,1010017,11,1,9,1676,A,1,16,1002,0.004980,0.008458,0.995349
3,0110065001,110065,306,Underwear,1010016,9,4,5,1339,B,1,61,1017,0.020219,0.025407,0.375479
4,0110065002,110065,306,Underwear,1010016,10,3,9,1339,B,1,61,1017,0.018205,0.025407,0.654917


In [131]:
winter_no = [252, 245, 71, 264, 262, 305, 80, 263, 496, 303, 349]
#     winter_no = [252, 245]
summer_no = [298, 59, 57, 255, 274, 91, 257, 299, 60]
#     summer_no = [255, 274, 257]

df['season'] = 0
df.loc[df['product_type_no'].isin(winter_no), 'season'] = -1
df.loc[df['product_type_no'].isin(summer_no), 'season'] = 1
df.head()

,article_id,product_code,product_type_no,product_group_name,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,department_no,index_code,index_group_no,section_no,garment_group_no,price_mean,price_max,mean_sales_channel_id,season
0,0108775015,108775,253,Garment Upper body,1010016,9,4,5,1676,A,1,16,1002,0.008142,0.009153,0.770778,0
1,0108775044,108775,253,Garment Upper body,1010016,10,3,9,1676,A,1,16,1002,0.008114,0.008508,0.710207,0
2,0108775051,108775,253,Garment Upper body,1010017,11,1,9,1676,A,1,16,1002,0.004980,0.008458,0.995349,0
3,0110065001,110065,306,Underwear,1010016,9,4,5,1339,B,1,61,1017,0.020219,0.025407,0.375479,0
4,0110065002,110065,306,Underwear,1010016,10,3,9,1339,B,1,61,1017,0.018205,0.025407,0.654917,0


In [132]:
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'], format="%Y-%m-%d")
transactions['timestamp'] = transactions.t_dat.values.astype(np.int64) // 10 ** 9 # 1585620000

agg_df = transactions.groupby('article_id')[['timestamp']].min()
agg_df.columns = ['min_timestamp']
agg_df.reset_index(drop=False, inplace=True)
agg_df.head()

,article_id,min_timestamp
0,0108775015,1537401600
1,0108775044,1537401600
2,0108775051,1537401600
3,0110065001,1537401600
4,0110065002,1537401600


In [133]:
df = df.merge(agg_df, on='article_id', how='left')
df.head()

,article_id,product_code,product_type_no,product_group_name,graphical_appearance_no,colour_group_code,perceived_colour_value_id,perceived_colour_master_id,department_no,index_code,index_group_no,section_no,garment_group_no,price_mean,price_max,mean_sales_channel_id,season,min_timestamp
0,0108775015,108775,253,Garment Upper body,1010016,9,4,5,1676,A,1,16,1002,0.008142,0.009153,0.770778,0,1.537402e+09
1,0108775044,108775,253,Garment Upper body,1010016,10,3,9,1676,A,1,16,1002,0.008114,0.008508,0.710207,0,1.537402e+09
2,0108775051,108775,253,Garment Upper body,1010017,11,1,9,1676,A,1,16,1002,0.004980,0.008458,0.995349,0,1.537402e+09
3,0110065001,110065,306,Underwear,1010016,9,4,5,1339,B,1,61,1017,0.020219,0.025407,0.375479,0,1.537402e+09
4,0110065002,110065,306,Underwear,1010016,10,3,9,1339,B,1,61,1017,0.018205,0.025407,0.654917,0,1.537402e+09


In [134]:
temp = df.rename(
    columns={'article_id': 'item_id:token', 'product_code': 'product_code:token', 'product_type_no': 'product_type_no:float',
             'product_group_name': 'product_group_name:token_seq', 'graphical_appearance_no': 'graphical_appearance_no:token', 
             'colour_group_code': 'colour_group_code:token', 'perceived_colour_value_id': 'perceived_colour_value_id:token', 
             'perceived_colour_master_id': 'perceived_colour_master_id:token', 'department_no': 'department_no:token', 
             'index_code': 'index_code:token', 'index_group_no': 'index_group_no:token', 'section_no': 'section_no:token', 
             'garment_group_no': 'garment_group_no:token', 'season': 'season:token', 'price_mean': 'price_mean:float', 'price_max': 'price_max:float', 'mean_sales_channel_id': 'mean_sales_channel_id:float',
            'min_timestamp': 'min_timestamp:float'})
temp.head()

,item_id:token,product_code:token,product_type_no:float,product_group_name:token_seq,graphical_appearance_no:token,colour_group_code:token,perceived_colour_value_id:token,perceived_colour_master_id:token,department_no:token,index_code:token,index_group_no:token,section_no:token,garment_group_no:token,price_mean:float,price_max:float,mean_sales_channel_id:float,season:token,min_timestamp:float
0,0108775015,108775,253,Garment Upper body,1010016,9,4,5,1676,A,1,16,1002,0.008142,0.009153,0.770778,0,1.537402e+09
1,0108775044,108775,253,Garment Upper body,1010016,10,3,9,1676,A,1,16,1002,0.008114,0.008508,0.710207,0,1.537402e+09
2,0108775051,108775,253,Garment Upper body,1010017,11,1,9,1676,A,1,16,1002,0.004980,0.008458,0.995349,0,1.537402e+09
3,0110065001,110065,306,Underwear,1010016,9,4,5,1339,B,1,61,1017,0.020219,0.025407,0.375479,0,1.537402e+09
4,0110065002,110065,306,Underwear,1010016,10,3,9,1339,B,1,61,1017,0.018205,0.025407,0.654917,0,1.537402e+09


In [135]:
# temp.to_csv(os.path.join(INTERMED_DIR, 'recbox_data.item'), index=False, sep='\t')
temp.to_csv('./recbox_data/recbox_data.item', index=False, sep='\t')
del temp, df, articles

# User

In [139]:
df = customers.drop(columns = ['FN', 'club_member_status', 'fashion_news_frequency', 'postal_code'])
df.head()

,customer_id,Active,age
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,49.0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,25.0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,24.0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,54.0
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,52.0


In [138]:
transactions = pd.read_csv(os.path.join(INPUT_DIR, 'transactions_train.csv'), dtype={"article_id": "str"})
transactions.sales_channel_id = transactions.sales_channel_id -1
agg_df = transactions.groupby('customer_id')[['sales_channel_id', 'price']].mean()
agg_df.columns = ['on_offline', 'price_level']
agg_df.reset_index(drop=False, inplace=True)
agg_df.head()

,customer_id,on_offline,price_level
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0.571429,0.030904
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.941860,0.030255
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,1.000000,0.039154
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,1.000000,0.030492
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0.846154,0.036130


In [140]:
df = df.merge(agg_df, on='customer_id', how='left')
df.head()

,customer_id,Active,age,on_offline,price_level
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,49.0,0.571429,0.030904
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,25.0,0.941860,0.030255
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,24.0,1.000000,0.039154
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,54.0,1.000000,0.030492
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,52.0,0.846154,0.036130


In [141]:
transactions = pd.read_csv(os.path.join(INPUT_DIR, 'transactions_train.csv'), dtype={"article_id": "str"})


In [142]:
articles = pd.read_csv(os.path.join(INPUT_DIR, 'articles.csv'), dtype={"article_id": "str"})

In [145]:
articles = articles[articles['index_group_name'].isin(['Ladieswear', 'Baby/Children', 'Menswear'])]

In [148]:
merged_df = transactions[['customer_id', 'article_id']].merge(articles[['article_id', 'index_group_name']], on='article_id', how='inner')
merged_df

,customer_id,article_id,index_group_name
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0663713001,Ladieswear
1,3681748607f3287d2c3a65e00bb5fb153de30e9becf158...,0663713001,Ladieswear
2,4ef5967ff17bf474bffebe5b16bd54878e1d4105f7b4ed...,0663713001,Ladieswear
3,6b7b10d2d47516c82a6f97332478dab748070f09693f09...,0663713001,Ladieswear
4,8ac137752bbe914aa4ae6ad007a9a0c5b67a1ab2b2d474...,0663713001,Ladieswear
...,...,...,...
23403657,ec794d6268ee3c75b5bbea014e3299d994bcd1180ecf6d...,0860135008,Menswear
23403658,ec794d6268ee3c75b5bbea014e3299d994bcd1180ecf6d...,0539060028,Baby/Children
23403659,ec794d6268ee3c75b5bbea014e3299d994bcd1180ecf6d...,0539060028,Baby/Children
23403660,ee9c048de16034be4aa95c91adce7f7bda442e6528406e...,0810737015,Baby/Children


In [149]:
agg_df = merged_df.groupby(['customer_id', 'index_group_name']).count()
agg_df.head()

article_id
customer_id                                        index_group_name            
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d... Baby/Children              5
                                                   Ladieswear                13
                                                   Menswear                   1
0000423b00ade91418cceaf3b26c6af3dd342b51fd051ee... Baby/Children              4
                                                   Ladieswear                62

In [150]:
agg_df = agg_df.reset_index(drop=False)
agg_df = pd.pivot_table(agg_df, index='customer_id', columns='index_group_name', values='article_id').fillna(0)
agg_df.head()

index_group_name,Baby/Children,Ladieswear,Menswear
customer_id,,,
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,5.0,13.0,1.0
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,4.0,62.0,0.0
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,0.0,12.0,4.0
00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a,0.0,11.0,0.0
0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d8cd0c725276a467a2a,0.0,2.0,0.0


In [151]:
agg_df.columns = ['children_count', 'ladieswear_count', 'menswear']
agg_df = agg_df.reset_index(drop=False)
agg_df.head()

,customer_id,children_count,ladieswear_count,menswear
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,5.0,13.0,1.0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,4.0,62.0,0.0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0.0,12.0,4.0
3,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0.0,11.0,0.0
4,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,0.0,2.0,0.0


In [152]:
df = df.merge(agg_df, on='customer_id', how='left')
df.head()

,customer_id,Active,age,on_offline,price_level,children_count,ladieswear_count,menswear
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,49.0,0.571429,0.030904,5.0,13.0,1.0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,25.0,0.941860,0.030255,4.0,62.0,0.0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,24.0,1.000000,0.039154,0.0,12.0,4.0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,54.0,1.000000,0.030492,NaN,NaN,NaN
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,52.0,0.846154,0.036130,0.0,11.0,0.0


In [153]:
df = df.fillna(0)

In [155]:
temp = df.rename(
    columns={'customer_id': 'user_id:token', 'age': 'age:float', 'Active': 'active:token', 'on_offline': 'on_offline:float', 'price_level': 'price_level:float',
            'children_count': 'children_count:float', 'ladieswear_count': 'ladieswear_count:float', 'menswear': 'menswear:float'})
temp.head()

,user_id:token,active:token,age:float,on_offline:float,price_level:float,children_count:float,ladieswear_count:float,menswear:float
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0.0,49.0,0.571429,0.030904,5.0,13.0,1.0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.0,25.0,0.941860,0.030255,4.0,62.0,0.0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0.0,24.0,1.000000,0.039154,0.0,12.0,4.0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0.0,54.0,1.000000,0.030492,0.0,0.0,0.0
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,52.0,0.846154,0.036130,0.0,11.0,0.0


In [156]:
temp.to_csv('./recbox_data/recbox_data.user', index=False, sep='\t')
del temp, df, customers

# Inter

In [86]:
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'], format="%Y-%m-%d")
transactions['timestamp'] = transactions.t_dat.values.astype(np.int64) // 10 ** 9 # 1585620000

In [87]:
# 1566620000: 2019-08-25
# 1585620000: 2020-04-01
temp = transactions[transactions['timestamp'] > 1566620000][['customer_id', 'article_id', 'timestamp']].rename(
    columns={'customer_id': 'user_id:token', 'article_id': 'item_id:token', 'timestamp': 'timestamp:float'})
temp.head()

,user_id:token,item_id:token,timestamp:float
15888555,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,0797492001,1566691200
15888556,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,0751551001,1566691200
15888557,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,0802703001,1566691200
15888558,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,0796314001,1566691200
15888559,00023e3dd8618bc63ccad995a5ac62e21177338d642d66...,0813991001,1566691200


In [88]:
# temp.to_csv(os.path.join(INTERMED_DIR, 'recbox_data.inter'), index=False, sep='\t')
temp.to_csv('./recbox_data/recbox_data.inter', index=False, sep='\t')
del temp

# Default

https://www.kaggle.com/code/tarique7/lb-0-0240-h-m-ensemble-magic-multi-blend

In [9]:
sub0 = pd.read_csv(os.path.join(INTERMED_DIR, 'submission00220.csv')).sort_values('customer_id').reset_index(drop=True)
sub1 = pd.read_csv(os.path.join(INTERMED_DIR, 'submission00227.csv')).sort_values('customer_id').reset_index(drop=True)
sub2 = pd.read_csv(os.path.join(INTERMED_DIR, 'submission00231.csv')).sort_values('customer_id').reset_index(drop=True)

sub0.shape, sub1.shape, sub2.shape

((1371980, 2), (1371980, 2), (1371980, 2))

In [10]:
sub0.columns = ['customer_id', 'prediction0']
sub0['prediction1'] = sub1['prediction']
sub0['prediction2'] = sub2['prediction']
del sub1, sub2
gc.collect()
sub0.head()

,customer_id,prediction0,prediction1,prediction2
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0751471001 0909370001 0915526001 09...,0568601043 0568601006 0745232001 0751471001 04...,0568601043 0568601006 0568601006 0568597006 04...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0924243001 0924243002 0923758001 0918522001 09...,0826211002 0739590027 0811835004 0764280001 07...,0826211002 0800436010 0739590027 0706016001 06...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0924243001 0924243002 0923758001 09...,0794321007 0858883002 0852643003 0727808007 08...,0794321007 0852643001 0852643003 0858883002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0924243001 0924243002 0923758001 0918522001 09...,0751471001 0579541001 0573085028 0673677002 06...,0448509014 0573085028 0751471001 0706016001 06...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0924243001 0924243002 0923758001 0918522001 09...,0730683050 0791587015 0896152002 0927530004 08...,0730683050 0791587015 0896152002 0818320001 09...


In [11]:
def cust_blend(dt, W = [1,1,1]):
    #Global ensemble weights
    #W = [1.15,0.95,0.85]
    
    #Create a list of all model predictions
    REC = []
    REC.append(dt['prediction0'].split())
    REC.append(dt['prediction1'].split())
    REC.append(dt['prediction2'].split())
    
    #Create a dictionary of items recommended. 
    #Assign a weight according the order of appearance and multiply by global weights
    res = {}
    for M in range(len(REC)):
        for n, v in enumerate(REC[M]):
            if v in res:
                res[v] += (W[M]/(n+1))
            else:
                res[v] = (W[M]/(n+1))
    
    # Sort dictionary by item weights
    res = list(dict(sorted(res.items(), key=lambda item: -item[1])).keys())
    
    # Return the top 12 itens only
    return ' '.join(res[:12])

sub0['prediction'] = sub0.apply(cust_blend, W = [1.05,1.00,0.95], axis=1)
sub0.head()

,customer_id,prediction0,prediction1,prediction2,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0751471001 0909370001 0915526001 09...,0568601043 0568601006 0745232001 0751471001 04...,0568601043 0568601006 0568601006 0568597006 04...,0568601043 0568601006 0751471001 0448509014 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0924243001 0924243002 0923758001 0918522001 09...,0826211002 0739590027 0811835004 0764280001 07...,0826211002 0800436010 0739590027 0706016001 06...,0826211002 0924243001 0739590027 0924243002 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0924243001 0924243002 0923758001 09...,0794321007 0858883002 0852643003 0727808007 08...,0794321007 0852643001 0852643003 0858883002 07...,0794321007 0858883002 0852643001 0852643003 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0924243001 0924243002 0923758001 0918522001 09...,0751471001 0579541001 0573085028 0673677002 06...,0448509014 0573085028 0751471001 0706016001 06...,0751471001 0448509014 0924243001 0573085028 05...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0924243001 0924243002 0923758001 0918522001 09...,0730683050 0791587015 0896152002 0927530004 08...,0730683050 0791587015 0896152002 0818320001 09...,0730683050 0924243001 0791587015 0896152002 09...


In [12]:
del sub0['prediction0']
del sub0['prediction1']
del sub0['prediction2']
gc.collect()
sub0.to_csv(os.path.join(INTERMED_DIR, 'submission.csv'), index=False)

In [13]:
del sub0
gc.collect()

0

# Model

In [5]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4RecF
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger

In [6]:
parameter_dict = {
    'data_path': './',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp'],
                 'item': ['item_id', 'product_code', 'product_type_no', 'product_group_name', 'graphical_appearance_no',
                      'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id',
                      'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no', 'season'],
                 'user': ['user_id', 'age']
             },
    'selected_features': ['product_code', 'product_type_no', 'product_group_name', 'graphical_appearance_no',
                          'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id',
                          'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no', 'season'],
    'neg_sampling': None,
    'epochs': 1,
    'eval_args': {
        'split': {'RS': [10, 0, 0]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'},
    'metrics': ['MAP'],
    'valid_metric': 'MAP@12',
    'topk': 12
}

config = Config(model='GRU4RecF', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
logger.info(config)

08 May 21:18    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ./recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 1
train_batch_size = 2048
learner = adam
learning_rate = 0.001
neg_sampling = None
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [10, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}
repeatable = True
metrics = ['MAP']
topk = [12]
valid_metric = MAP@12
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separator = 	
seq_separator =  
USER_ID_FIELD = user_id
ITEM_ID_FIELD = item_id
RATING_FIELD = rating
TIME_FIELD = timestamp
seq_len = None
LABEL_FIELD = label
threshold =

In [7]:
dataset = create_dataset(config)
logger.info(dataset)

08 May 21:19    INFO  recbox_data
The number of users: 15459
Average actions of users: 59.21956268598784
The number of items: 7330
Average actions of items: 124.9032610178742
The number of inters: 915416
The sparsity of the dataset: 99.19214553975321%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'age', 'product_code', 'product_type_no', 'product_group_name', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id', 'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no']
recbox_data
The number of users: 15459
Average actions of users: 59.21956268598784
The number of items: 7330
Average actions of items: 124.9032610178742
The number of inters: 915416
The sparsity of the dataset: 99.19214553975321%
Remain Fields: ['user_id', 'item_id', 'timestamp', 'age', 'product_code', 'product_type_no', 'product_group_name', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_mas

In [8]:
train_data, valid_data, test_data = data_preparation(config, dataset)
# model loading and initialization
model = GRU4RecF(config, train_data.dataset).to(config['device'])
logger.info(model)
# trainer loading and initialization
trainer = Trainer(config, model)

08 May 21:19    INFO  [Training]: train_batch_size = [2048] negative sampling: [None]
[Training]: train_batch_size = [2048] negative sampling: [None]
[Training]: train_batch_size = [2048] negative sampling: [None]
08 May 21:19    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [10, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [10, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [10, 0, 0]}, 'group_by': 'user', 'order': 'TO', 'mode': 'full'}]
08 May 21:19    INFO  GRU4RecF(
  (item_embedding): Embedding(7330, 64, padding_idx=0)
  (feature_embed_layer): FeatureSeqEmbLayer(
    (token_embedding_table): ModuleDict(
      (item): FMEmbedding(
        (embedding): Embedding(3935, 64)
      )
    )
    (float_embedding_table): ModuleDict(
      (item): Embedding(1, 64)
    )
    (token_seq_embedding_table):

In [ ]:
# model training
best_valid_score, best_valid_result = trainer.fit(train_data)

In [ ]:
from recbole.utils.case_study import full_sort_topk
external_user_ids = dataset.id2token(
    dataset.uid_field, list(range(dataset.user_num)))[1:]#fist element in array is 'PAD'(default of Recbole) ->remove it 

In [ ]:
import torch
from recbole.data.interaction import Interaction

def add_last_item(old_interaction, last_item_id, max_len=50):
    new_seq_items = old_interaction['item_id_list'][-1]
    if old_interaction['item_length'][-1].item() < max_len:
        new_seq_items[old_interaction['item_length'][-1].item()] = last_item_id
    else:
        new_seq_items = torch.roll(new_seq_items, -1)
        new_seq_items[-1] = last_item_id
    return new_seq_items.view(1, len(new_seq_items))

def predict_for_all_item(external_user_id, dataset, model):
    model.eval()
    with torch.no_grad():
        uid_series = dataset.token2id(dataset.uid_field, [external_user_id])
        index = np.isin(dataset.inter_feat[dataset.uid_field].numpy(), uid_series)
        input_interaction = dataset[index]
        test = {
            'item_id_list': add_last_item(input_interaction, 
                                          input_interaction['item_id'][-1].item(), model.max_seq_length),
            'item_length': torch.tensor(
                [input_interaction['item_length'][-1].item() + 1
                 if input_interaction['item_length'][-1].item() < model.max_seq_length else model.max_seq_length])
        }
        new_inter = Interaction(test)
        new_inter = new_inter.to(config['device'])
        new_scores = model.full_sort_predict(new_inter)
        new_scores = new_scores.view(-1, test_data.dataset.item_num)
        new_scores[:, 0] = -np.inf  # set scores of [pad] to -inf
    return torch.topk(new_scores, 12)

In [ ]:
predict_for_all_item('0109ad0b5a76924a1b58be677409bb601cc8bead9a87b8ce5b08a4a1f5bc71ef', 
                     dataset, model)

In [160]:
j = {'ok': 1}
jj = {}
'ok' in jj.keys()

False

In [158]:
j.keys() in 'ok'

TypeError: 'in <string>' requires string as left operand, not dict_keys